<a href="https://colab.research.google.com/github/JuanJoseMV/neuraltextgen/blob/main/Finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installations
## Requirements

In [1]:
%%capture
!git clone --recursive https://github.com/JuanJoseMV/neuraltextgen.git
!pip install -r /content/neuraltextgen/texygen/requirements.txt
!pip install simpletransformers

In [17]:
import torch

import numpy as np
import re
import pandas as pd
import os
os.chdir("/content/neuraltextgen/")

## Nvidia APEX

In [ ]:
%%capture
!pip install torch==1.7.1+cu101 torchvision==0.8.2+cu101 torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
%%writefile setup.sh
export CUDA_HOME=/usr/local/cuda-10.1
git clone https://github.com/NVIDIA/apex
pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex

Writing setup.sh


In [ ]:
%%capture
!sh setup.sh

# Finetuning




## Example 1 - Motivational quotes
The first experiment is focused on showing the functionality of the fine-tuning of BertTextGenerator.
To do this we will finetune an italian model on a small set of Italian motivational quotes, to try make him generate new ones.




### Dataset creation
We can start creating our dataset. To do this we will scrape some sites in order to gather a good amount of quotes.

The first site we used is https://www.frasimania.it/frasi-motivazionali/, that contains 125 quotes as:

>Prima ti ignorano, poi ti deridono, poi ti combattono. Poi vinci.
(Mahatma Gandhi)

In [3]:
import pandas as pd
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup as soup

url = "https://www.frasimania.it/frasi-motivazionali/"
req = Request(url , headers={'User-Agent': 'Mozilla/5.0'})
webpage = urlopen(req).read()
page_soup = soup(webpage, "html.parser")

In [5]:
import re
with open('/content/quotes.txt', 'w') as f:
  containers = page_soup.findAll('p', attrs={'class':'singola'})
  for container in containers:
    quote = container.getText()
    quote = quote.replace('\n', ' ')
    
    quote, author = quote.split('(')
    author = author.split(',')[0]
    f.write(f'{quote.strip()} - ({author.strip()}\n')


The second site is "https://aforisticamente.com/le-piu-belle-frasi-motivazionali/. Here there are 100 quotes as:
> Abbiamo quaranta milioni di ragioni per fallire, ma non una sola scusa.
(Rudyard Kipling)



In [6]:
import pandas as pd
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup as soup

url = "https://aforisticamente.com/le-piu-belle-frasi-motivazionali/"
req = Request(url , headers={'User-Agent': 'Mozilla/5.0'})
webpage = urlopen(req).read()
page_soup = soup(webpage, "html.parser")


In [7]:
with open('/content/quotes.txt', 'a') as f:
  containers = page_soup.findAll('p')
  for container in containers[4:-10]:
    quote = container.getText()
    quote = quote.replace('\n', ' ')

    if len(quote) < 5:
      pass
    else:
      quote, author = quote.split('(')
      author = author.split(',')[0]
      f.write(f'{quote.strip()} - ({author.strip()}\n')

The third site is https://www.oberlo.it/blog/frasi-motivazionali. Here there are 200 quotes. The quotes are structured a bit differently than in the previous examples
>4. “Se le tue azioni ispirano gli altri a sognare di più, a imparare di più, a fare di più e a diventare di più, sei un leader.” — John Quincy Adams, 6° Presidente degli Stati Uniti (Clicca per twittare)

There is a number prepended, a description of the author plus some other noise text that can ber removed

In [8]:

import pandas as pd
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup as soup

url = "https://www.oberlo.it/blog/frasi-motivazionali"
req = Request(url , headers={'User-Agent': 'Mozilla/5.0'})
webpage = urlopen(req).read()
page_soup = soup(webpage, "html.parser")

In [9]:
containers = page_soup.findAll('p')
text = [container.getText() for container in containers[:245]]

In [12]:
import re

start = re.compile('^[\d]+.')
with open('/content/quotes.txt', 'a') as f:
  for string in text:
    matches = start.search(string) #only lines that start with a number
    if matches:
      try:
        string = string[matches.end()+1:]
        quote, author = string.split('—')
        # quote = quote.replace('“','”')
        quote = quote.replace('“','')
        quote = quote.replace('”', '')
        author = author.split(',')[0]
        f.write(f'{quote.strip()} - ({author.strip()})\n')
      except:
        pass

In [13]:
import pandas as pd
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup as soup

url = "https://www.esserealtop.it/365-frasi-motivazionali-che-ti-cambieranno-la-vita/"
req = Request(url , headers={'User-Agent': 'Mozilla/5.0'})
webpage = urlopen(req).read()
page_soup = soup(webpage, "html.parser")

In [14]:
containers = page_soup.findAll('p')
text = [container.getText() for container in containers]

In [ ]:
with open('quotes.txt', 'a') as f:
  for string in text:
    if string.startswith('“'):
  
      string = string.replace('”', '“')
      string = string.replace('”', '')
      string = string.replace('“', '')
      # string = string.replace('\xa0', ' ')
      
      try:
        quote, author = string.split('–')
        # author = author.split(',')[0]
        f.write(f'{quote.strip()} - ({author.strip()})\n')
        # f.write(splitted[0] + ' (' + author + ' )\n')
      except:
        quote = string;
        author = 'anonimo'
        f.write(f'{quote.strip()} - {author.strip()}\n')
        
      

In [16]:
for line in open('quotes.txt').readlines()[:10]:
  print(line)

Prima ti ignorano, poi ti deridono, poi ti combattono. Poi vinci. - (Mahatma Gandhi)

Ho sempre tentato. Ho sempre fallito. Non discutere. Prova ancora. Fallisci ancora. Fallisci meglio. - (Samuel Beckett)

Se vuoi qualcosa che non hai mai avuto, devi fare qualcosa che non hai mai fatto. - (Thomas Jefferson)

Ci sono due regole nella vita: 1. Non mollare mai; 2. Non dimenticare mai la regola n° 1. - (Duke Ellington)

La vita è per il 10% cosa ti accade e per il 90% come reagisci. - (Charles R. Swindoll)

Non si è mai troppo vecchi per fissare un nuovo obiettivo o per sognare un nuovo sogno. - (C.S. Lewis)

Non rinunciare a provare a fare ciò che vuoi veramente fare. Dove c’è amore e ispirazione, non credo che si possa sbagliare. - (Ella Fitzgerald)

Due strade divergevano nel bosco, ed io… io scelsi quella meno battuta e questo fece la differenza. - (Robert Frost)

Sono grato a tutte quelle persone che mi hanno detto di no. È grazie a loro se sono quel che sono. - (Albert Einstein)

Il

### Model
Now that our set of quotes is ready we have practically done. It remains only to chose a pretrained BERT model and initliaze the BertTextGenerator. We will use this model https://huggingface.co/dbmdz/bert-base-italian-cased

In [53]:
from NeuralTextGenerator import BertTextGenerator

it_bert_model = BertTextGenerator("dbmdz/bert-base-italian-uncased")

Some weights of the model checkpoint at dbmdz/bert-base-italian-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [54]:
lines = open('/content/quotes.txt').readlines()
it_bert_model.finetune(lines, epochs=4, batch_size=32,optimizer_parameters = dict(lr=5e-5))


======== Epoch 1 / 4 ========
Training...

  Average training loss: 2.83
  Training epcoh took: 0:00:34

======== Epoch 2 / 4 ========
Training...

  Average training loss: 0.42
  Training epcoh took: 0:00:34

======== Epoch 3 / 4 ========
Training...

  Average training loss: 0.33
  Training epcoh took: 0:00:34

======== Epoch 4 / 4 ========
Training...

  Average training loss: 0.29
  Training epcoh took: 0:00:34

Training complete!
Total training took 0:02:15 (h:mm:ss)


In [57]:
parameters = {'n_sentences': 3, 
              'batch_size': 1, 
              'avg_len':25, 
              'std_len':5,
              'generation_method':'parallel',
              'max_iter':500,
              'burnin':450,
              }

sents = it_bert_model.generate(save_to_path='/content/dante_generated.txt', **parameters)

for s in sents:
    print(s)
    

carmen sevilla ( siviglia ), siviglia ( ana aquis ), renata berry ( lisbona ), sydney ( sydney )
covent laurent favre ( oscar goethe ) jane fonda ( putin )
! ) ( anonimo ) sto giocando con i delfini..!! ( anonimo ) ( rifugiato in siria )


In [55]:
parameters = {'n_sentences': 4, 
              'batch_size': 1, 
              'avg_len':25, 
              # 'std_len':5,
              'generation_method':'parallel',
              'max_iter':500,
              # 'burnin':330,
              }

sents = it_bert_model.generate(save_to_path='/content/quotes_parallel_generation.txt', **parameters)

for s in sents:
    print(s)

perche essere cosi ingenuo e no forse uno dei due non si ama lo stesso. ( henry franklin ) ( henry forde universita harvard )
( ( ricorda cosa ti ho detto ) ) vincerai. ( anonimo ) ( arthur farioski ) ( henry wells ) ( anonimo )
una vita poetica che ricorda le fiabe cosa vedere atena sheila ( film ) atena sheila ( piede )
i juan la fortuna ( hernan camacho ), i juan la fama ( osho gosseto ), juan la fama ( paulo munari )


In [56]:
parameters = {'n_sentences': 4, 
              'batch_size': 1, 
              'avg_len':25, 
              'std_len':5,
              'generation_method':'attention',
              'max_iter':500,
              'burnin':330,
              }

sents = it_bert_model.generate(save_to_path='/content/quotes_attention_generation.txt', **parameters)

for s in sents:
    print(s)

( john s. osborne brown ) ( san francisco ). ( l ’ autore e henry button ).
(... )... (... ) ( michael schaferman )
( ( huai ) ) ( ( huai ) ) ( huai ) ) ( ( huai ) ) ( ( hua ) ( ) )
non potevamo separarci. potevamo... rimanere uniti. ( adam kaczynski )


In [ ]:
parameters = {'n_sentences': 8, 
              'batch_size': 1, 
              'avg_len':25, 
              'std_len':5,
              'generation_method':'attention',
              'max_iter':500,
              'burnin':330,
              }

sents = it_bert_model.generate(save_to_path='/content/dante_generated.txt', **parameters)

for s in sents:
    print(s)

se dirigi gli affari potresti non poter avere le raffinerie. ( j. schuller ) anche se si hanno solo le raffinerie non puoi disertare.
se vuoi davvero fare una cosa, e meglio se la smetti di farla. ( thomas edison )
quello che succede e che vinciamo e cadiamo fino al risultato finale ( anche se cadiamo come cagnetti )
( se non vuoi essere con un cucchiaino in mano ), devi concentrarti su chi vuoi veramente essere. ( ra )


In [ ]:
parameters = {'n_sentences': 4, 
              'batch_size': 1, 
              'avg_len':25, 
              'std_len':5,
              'generation_method':'attention',
              'max_iter':500,
              'burnin':430,
              }

sents = it_bert_model.generate(save_to_path='/content/dante_generated.txt', **parameters)

for s in sents:
    print(s)

a scrivere e a leggere le parole. ( anonimo )... da dove iniziare?
[ j. k. ] " so solo che sto lavorando per vincere. " fidati, vincerai. ( henry j. d williamson ) ( anonimo )
essere terrorizzato da freud fino al suo scopo. ma se devo lavorare tutta la notte per il mio scopo con esso devo lavorare tutta la notte per farlo.
quando muori, la vita finisce. non ci resta che aspettare finche non muori, perche non sai cosa ti aspetta ( anonimo )


In [ ]:
parameters = {'n_sentences': 4, 
              'batch_size': 1, 
              'avg_len':25, 
              'std_len':5,
              'generation_method':'sequential',
              'max_iter':500,
              'burnin':430,
              }

sents = it_bert_model.generate(save_to_path='/content/dante_generated.txt', **parameters)

for s in sents:
    print(s)

niente e meglio di fare quello che ti pare. e poi, ti sto dicendo che sei grande abbastanza.
non hai nulla da fare, tutti i giorni, se non stai cercando di fare tutto cio che e necessario. ( norman bates )
lo ho fatto perche sapevo che dovevo diventare uno, che lo avevo gia fatto e che avrei fallito quando ho capito che dovevo diventare uno.
non e questo l ’ obiettivo a cui in realta sto mirando, sebbene sia quello che cerco sempre di raggiungere. ( joshua reynolds )


In [ ]:
parameters = {'n_sentences': 3, 
              'batch_size': 1, 
              'avg_len':25, 
              'std_len':5,
              'generation_method':'parallel',
              'max_iter':500,
              'burnin':450,
              }

sents = it_bert_model.generate(save_to_path='/content/dante_generated.txt', **parameters)

for s in sents:
    print(s)
    

faro del mio meglio per non essere trattato come un fallito. ma finche saro fallito, non lo faro. ( ripeto, ingra ) ( frank lloyd wright )
provaci ( anche se non cadi ) sii un eroe. sii forte. sii coraggioso. sii un eroe.
se poi ti rifiuti di fare cose buone, ti fermi per sempre. ( smetti di cercare di fare le cose che vuoi fare. )


In [ ]:
parameters = {'n_sentences': 3, 
              'batch_size': 1, 
              'avg_len':25, 
              'std_len':5,
              'generation_method':'parallel',
              'max_iter':500
              }

sents = it_bert_model.generate(save_to_path='/content/dante_generated.txt', **parameters)

for s in sents:
    print(s)
    

mi piacerebbe trovare un nuovo modo per prepararmi a questa nuova sfida.
no, vattene via, paul berry! ( paul berry che canta ) ( peter t. dieter ).
moezsen sta provando a diventare ’ li ’ dentro. se stro qualcosa, ‘ allora si manifesta la sua trascendenza ’.


In [4]:
x = [       "se dirigi gli affari potresti non poter avere le raffinerie. ( j. schuller )\n",
            "se vuoi davvero fare una cosa, e meglio se la smetti di farla. ( thomas edison )\n",
            "(... )... (... ) ( michael schaferman )\n",
            "( se non vuoi essere con un cucchiaino in mano ), devi concentrarti su chi vuoi veramente essere. ( ra )\n",
            "a scrivere e a leggere le parole. ( anonimo )... da dove iniziare?\n",
            "( ( huai ) ) ( ( huai ) ) ( huai ) ) ( ( huai ) ) ( ( hua ) ( ) )\n",
            "[ j. k. ] \" so solo che sto lavorando per vincere. \" fidati, vincerai. ( henry j. d williamson ) ( anonimo )\n",
            "quando muori, la vita finisce. non ci resta che aspettare finche non muori, perche non sai cosa ti aspetta ( anonimo )\n",
            "faro del mio meglio per non essere trattato come un fallito. ma finche saro fallito, non lo faro. ( ripeto, ingra ) ( frank lloyd wright )\n",
            "provaci ( anche se non cadi ) sii un eroe. sii forte. sii coraggioso. sii un eroe.\n",
            "se poi ti rifiuti di fare cose buone, ti fermi per sempre. ( smetti di cercare di fare le cose che vuoi fare. )\n",
            "mi piacerebbe trovare un nuovo modo per prepararmi a questa nuova sfida.\n",
            "no, vattene via, paul berry! ( paul berry che canta ) ( peter t. dieter ).\n",
            "moezsen sta provando a diventare ’ li ’ dentro. se stro qualcosa, ‘ allora si manifesta la sua trascendenza ’.\n",
            "niente e meglio di fare quello che ti pare. e poi, ti sto dicendo che sei grande abbastanza.\n",
            "non hai nulla da fare, tutti i giorni, se non stai cercando di fare tutto cio che e necessario. ( norman bates )\n",
            "lo ho fatto perche sapevo che dovevo diventare uno, che lo avevo gia fatto e che avrei fallito quando ho capito che dovevo diventare uno.\n",
            "non e questo l ’ obiettivo a cui in realta sto mirando, sebbene sia quello che cerco sempre di raggiungere. ( joshua reynolds )\n"
            ]

for s in x:
  print(s, end='')



se dirigi gli affari potresti non poter avere le raffinerie. ( j. schuller )
se vuoi davvero fare una cosa, e meglio se la smetti di farla. ( thomas edison )
(... )... (... ) ( michael schaferman )
( se non vuoi essere con un cucchiaino in mano ), devi concentrarti su chi vuoi veramente essere. ( ra )
a scrivere e a leggere le parole. ( anonimo )... da dove iniziare?
( ( huai ) ) ( ( huai ) ) ( huai ) ) ( ( huai ) ) ( ( hua ) ( ) )
[ j. k. ] " so solo che sto lavorando per vincere. " fidati, vincerai. ( henry j. d williamson ) ( anonimo )
quando muori, la vita finisce. non ci resta che aspettare finche non muori, perche non sai cosa ti aspetta ( anonimo )
faro del mio meglio per non essere trattato come un fallito. ma finche saro fallito, non lo faro. ( ripeto, ingra ) ( frank lloyd wright )
provaci ( anche se non cadi ) sii un eroe. sii forte. sii coraggioso. sii un eroe.
se poi ti rifiuti di fare cose buone, ti fermi per sempre. ( smetti di cercare di fare le cose che vuoi fare. )
mi

In [6]:
x = [
            "\n",
            " che mai non m ’ io coglia ; \n",
            " e non abbracci che li si ridesse, \n",
            " che piu tempo fece prodente. \n",
            "\n",
            "\n",
            " ma vidi io di lo vento che \n",
            " del son s ’ accesi io : \n",
            " per tanto piacer che questo venisse il triscio ». \n",
            "\n",
            "\n",
            " ma gia di queste le zanne \n",
            " sonn piu, si malicce, \n",
            " che prima l ’ occhio non era in ciel ; \n",
            "\n",
            "\n",
            " vedi, se tu vedi, che la terra s ’ arresta, \n",
            " a te e a questo mondo \n",
            " c ’ e ’ l mondo che si drizza ». \n",
            "\n",
            "e tornar per le foci richiudendo la gola, \n",
            " si che non se ne andassero, \n",
            " mi volsi al petto e a li squarci. \n",
            "\n",
            "e io, che mi convivo \n",
            " dentro e fuori, venia a me, \n",
            " e vidi la prima vista \n",
            " che la mente m ’ avea piu piena. \n",
            "\n",
            "venne poi innanzi a me la mia mente, \n",
            " per molti anni, e non fu, \n",
            " benche non fosse, \n",
            " che non fosse in me alcuna virtu ». \n",
            "\n",
            "e disse a me : ‘ e tu? ’ \n",
            " piu in la che vidi, piu non si facea ; \n",
            " questo e ’ l padre mio, che e con me ». \n",
            "\n",
            "\n",
            " e cato e cato, e in lui \n",
            " si fece ’ l uno e un altro, \n",
            " si che ’ l le membra e lo spirito movea ». \n",
            "\n",
            "\n",
            " sappi pero che la tua mente e buona, \n",
            " che si fa la volonta di dio, \n",
            " com ’ e stata fatta l ’ arte del mondo, \n",
            " che non e morta. \n",
            "\n",
            "“ e ’ l viso mio “ chinato ”, \n",
            " e ’ l vidi correr dietro a me, \n",
            " si tentenno e disse : « loco, loco! ».\n",
          ]
for s in x:
  print(s, end='')


 che mai non m ’ io coglia ; 
 e non abbracci che li si ridesse, 
 che piu tempo fece prodente. 


 ma vidi io di lo vento che 
 del son s ’ accesi io : 
 per tanto piacer che questo venisse il triscio ». 


 ma gia di queste le zanne 
 sonn piu, si malicce, 
 che prima l ’ occhio non era in ciel ; 


 vedi, se tu vedi, che la terra s ’ arresta, 
 a te e a questo mondo 
 c ’ e ’ l mondo che si drizza ». 

e tornar per le foci richiudendo la gola, 
 si che non se ne andassero, 
 mi volsi al petto e a li squarci. 

e io, che mi convivo 
 dentro e fuori, venia a me, 
 e vidi la prima vista 
 che la mente m ’ avea piu piena. 

venne poi innanzi a me la mia mente, 
 per molti anni, e non fu, 
 benche non fosse, 
 che non fosse in me alcuna virtu ». 

e disse a me : ‘ e tu? ’ 
 piu in la che vidi, piu non si facea ; 
 questo e ’ l padre mio, che e con me ». 


 e cato e cato, e in lui 
 si fece ’ l uno e un altro, 
 si che ’ l le membra e lo spirito movea ». 


 sappi pero che la tua mente 

## Example 2 - Dante's Divine Comedy

In this example we will train our own neural-poet!
As before we will start from an italian model https://huggingface.co/dbmdz/bert-base-italian-uncased.


In [ ]:
it_bert_model = BertTextGenerator("dbmdz/bert-base-italian-uncased")

Downloading:   0%|          | 0.00/433 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/442M [00:00<?, ?B/s]

Some weights of the model checkpoint at dbmdz/bert-base-italian-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/243k [00:00<?, ?B/s]

Then we need Dante's Divine Comedy. We can download it from this repo (https://github.com/dlang/druntime.git)

In [ ]:
!curl https://raw.githubusercontent.com/dlang/druntime/master/benchmark/extra-files/dante.txt > dante.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  543k  100  543k    0     0  6252k      0 --:--:-- --:--:-- --:--:-- 6252k


In [ ]:
path_dante = '/content/dante.txt'

with open(path_dante, 'r') as f_dante:
  i = 0
  while i < 30:
    print(f_dante.readline(), end='')
    i+=1

LA DIVINA COMMEDIA
di Dante Alighieri





INFERNO




Inferno · Canto I


Nel mezzo del cammin di nostra vita
mi ritrovai per una selva oscura,
ché la diritta via era smarrita.

Ahi quanto a dir qual era è cosa dura
esta selva selvaggia e aspra e forte
che nel pensier rinova la paura!

Tant’ è amara che poco è più morte;
ma per trattar del ben ch’i’ vi trovai,
dirò de l’altre cose ch’i’ v’ho scorte.

Io non so ben ridir com’ i’ v’intrai,
tant’ era pien di sonno a quel punto
che la verace via abbandonai.


Above are printed the first 4 tercets of Dante's masterpiece. The Divine Comedy is divided in three cantiche: Inferno (Hell), Purgatorio (Purgatory), and Paradiso (Paradise) – each consisting of 33 cantos.

Tipically a model is trained on a text file considering all the lines on at time, however as we can see this file contains a lot of empty lines and intestations. Moerover considering a line at time we break the tercets, that are the 3-lines structure of which the poem is composed.

Our goal is to train a model able to produce small poems. So a better idea would be to clean the file and consider tercets instead of single lines.

To do this it can come in handy the `Formatter` class present in  `NeuralTextGenerator.textprocessing` with which we can define the pattern for the lines that we want to extract. Moreover it allows to define a set of tokens that we want to preserve before and after the tokenization. In this case for example, to let BERT learn the tercet structure is important that it learns also where to place the newlines ('\n') characters. To do this we simply need to specify a list of `replace_tokens` and define the `unused_type`. This can be the most tricky part since there is not a conventional way and each BERT model can use a different set of tokens. For example 'bert-base-uncased' used '`[unused-1], [unused-2], etc...` while 'bert-base-italian-uncased' (the model we are using) uses `unused1, unused2, etc...`

Note:
Each token in the `replace_tokens` is replaced with an unused token before the tokenization and then replaced back after the generation of the text

In [ ]:
from neuraltextgen.textprocessing import Formatter

tercet_pattern = '\n((.)+\n){3}'
formatter = Formatter(replace_tokens=['\n'], unused_type='unusedi')
lines = formatter.format(path_dante, pattern = tercet_pattern)

for i,l in enumerate(lines[:5]):
  print(f"Tercet {i+1}: {l}")

Tercet 1:  unused1 Nel mezzo del cammin di nostra vita unused1 mi ritrovai per una selva oscura, unused1 ché la diritta via era smarrita. unused1 
Tercet 2:  unused1 Ahi quanto a dir qual era è cosa dura unused1 esta selva selvaggia e aspra e forte unused1 che nel pensier rinova la paura! unused1 
Tercet 3:  unused1 Tant’ è amara che poco è più morte; unused1 ma per trattar del ben ch’i’ vi trovai, unused1 dirò de l’altre cose ch’i’ v’ho scorte. unused1 
Tercet 4:  unused1 Io non so ben ridir com’ i’ v’intrai, unused1 tant’ era pien di sonno a quel punto unused1 che la verace via abbandonai. unused1 
Tercet 5:  unused1 Ma poi ch’i’ fui al piè d’un colle giunto, unused1 là dove terminava quella valle unused1 che m’avea di paura il cor compunto, unused1 


As we can see now the tercets have been transformed into simple lines and the '\n' characters have been replaced with 'unused1' so that the '\n' are not removed by the tokenizer and BERT will be able to learn the position of 'unused1' in the text.

Finally we need to finetune the model. This is very simple, we only need to pass the lines that we have gathered before and if we want specify the hyperparameters of the finetuning

In [ ]:
it_bert_model.finetune(lines, epochs=4, optimizer_parameters = dict(lr=5e-5))


======== Epoch 1 / 4 ========
Training...
  Batch    25  of    148.    Elapsed: 0:00:21.
  Batch    50  of    148.    Elapsed: 0:00:41.
  Batch    75  of    148.    Elapsed: 0:01:02.
  Batch   100  of    148.    Elapsed: 0:01:22.
  Batch   125  of    148.    Elapsed: 0:01:43.

  Average training loss: 2.60
  Training epcoh took: 0:02:01

======== Epoch 2 / 4 ========
Training...
  Batch    25  of    148.    Elapsed: 0:00:20.
  Batch    50  of    148.    Elapsed: 0:00:41.
  Batch    75  of    148.    Elapsed: 0:01:01.
  Batch   100  of    148.    Elapsed: 0:01:21.
  Batch   125  of    148.    Elapsed: 0:01:42.

  Average training loss: 2.01
  Training epcoh took: 0:02:00

======== Epoch 3 / 4 ========
Training...
  Batch    25  of    148.    Elapsed: 0:00:20.
  Batch    50  of    148.    Elapsed: 0:00:41.
  Batch    75  of    148.    Elapsed: 0:01:01.
  Batch   100  of    148.    Elapsed: 0:01:21.
  Batch   125  of    148.    Elapsed: 0:01:42.

  Average training loss: 1.89
  Training 

Now the fun part!
Our model is finetuned and ready to generate some beautiful poems.


In [ ]:
parameters = {'n_sentences': 4, 
              'batch_size':4, 
              'avg_len':30, 
              'std_len':4,
              'generation_method':'attention',
              'max_iter':200
              }

sents = it_bert_model.generate(save_to_path='/content/dante_generated.txt', **parameters)

for s in formatter.unformat(sents):
    print(s)
    


 che mai non m ’ io coglia ; 
 e non abbracci che li si ridesse, 
 che piu tempo fece prodente. 


 ma vidi io di lo vento che 
 del son s ’ accesi io : 
 per tanto piacer che questo venisse il triscio ». 


 ma gia di queste le zanne 
 sonn piu, si malicce, 
 che prima l ’ occhio non era in ciel ; 


 
 fu tra li occhi contenti 
 e vidi 
 tutti si tra loro cominciar in dietro 
 feretro a e fuor d ’ orbi. 



In [ ]:
parameters = {'n_sentences': 4, 
              'batch_size':4, 
              'avg_len':30, 
              'std_len':4,
              'generation_method':'attention',
              'max_iter':150
              }

sents = it_bert_model.generate(save_to_path='/content/dante_generated.txt', **parameters)

for s in formatter.unformat(sents):
    print(s)
    

al foco di lui perde voce, che ponzio, fama per cui poeta 
 non puo fregiarla o sua pena. 


 si ’ m terra in la dove si parte, fa ragion e varia, 
 di grazia e suo tempso sapea : 


 da come m ’ e noto 
 che del mio caro vien quel convien, 
 da quel che pur e pur, 

sappi che l ’ arte mia e si santa 
 per fare la guerra che la coscienza 
 gia har cristo in mano la mente. 



In [ ]:
%%time
parameters = {'n_sentences': 4, 
              'batch_size':4, 
              'avg_len':35, 
              'generation_method':'sequential',
              'max_iter':500,
              'burnin':400,
              'top_k':100
              }

sents = it_bert_model.generate(save_to_path='/content/dante_generated.txt', **parameters)

for s in formatter.unformat(sents):
    print(s)


 vedi, se tu vedi, che la terra s ’ arresta, 
 a te e a questo mondo 
 c ’ e ’ l mondo che si drizza ». 

e tornar per le foci richiudendo la gola, 
 si che non se ne andassero, 
 mi volsi al petto e a li squarci. 

e io, che mi convivo 
 dentro e fuori, venia a me, 
 e vidi la prima vista 
 che la mente m ’ avea piu piena. 

venne poi innanzi a me la mia mente, 
 per molti anni, e non fu, 
 benche non fosse, 
 che non fosse in me alcuna virtu ». 

CPU times: user 22 s, sys: 36.1 ms, total: 22 s
Wall time: 22 s


In [ ]:
%%time
parameters = {'n_sentences': 4, 
              'batch_size':4, 
              'avg_len':30, 
              'generation_method':'sequential',
              'max_iter':500,
              'burnin':400,
              'top_k':100
              }

sents = it_bert_model.generate(save_to_path='/content/dante_generated.txt', **parameters)

for s in formatter.unformat(sents):
    print(s)


 ov ’ aprii li occhi, la tua vista, e la tua vedetta, 
 vidi il sole e la luce del mondo che ti guarda. 


 e per la prima volta 
 queste son le genti che son vicine, 
 nulla e nullo a quel che non si disgiunge ». 


 per far si che, di volto in volto, 
 dietro al viso mio mi guardava, 
 poco a poco mi venisse a seguir umilmente. 

« non fui di quella donna, ma quella che s ’ empia, 
 che per la sua vita avea di fronte a se la vergogna ». 

CPU times: user 21.6 s, sys: 48.3 ms, total: 21.6 s
Wall time: 21.6 s


In [ ]:

%%time
parameters = {'n_sentences': 4, 
              'batch_size':4, 
              'avg_len':30, 
              'generation_method':'attention',
              'max_iter':1000,
              'burnin':850,
              'top_k':100
              }

sents = it_bert_model.generate(save_to_path='/content/dante_generated.txt', **parameters)

for s in formatter.unformat(sents):
    print(s)

disse : « questo e tempo di tenebra 
 di viver e di morte 
 si che si vive e si si che si vive ». 

e io m ’ unisco ad esse, 
 si ch ’ a creder mi fa creder, 
 e mi fa lor sembiante. 


 non perdendo in me la paura 
 che, per quanto mi dolse, 
 pur di non aver paura mi rispuose. 


 ma se tu credi a me, crederai 
 a la mia fede, 
 non a la compagnia, ma a creder meco. 

CPU times: user 33.6 s, sys: 61 ms, total: 33.6 s
Wall time: 33.5 s


In [ ]:
# Don't delete
%%time
parameters = {'n_sentences': 4, 
              'batch_size':4, 
              'avg_len':35, 
              'generation_method':'attention',
              'max_iter':500,
              'burnin':350,
              'top_k':100
              }

sents = it_bert_model.generate(save_to_path='/content/dante_generated.txt', **parameters)

for s in formatter.unformat(sents):
    print(s)

e disse a me : ‘ e tu? ’ 
 piu in la che vidi, piu non si facea ; 
 questo e ’ l padre mio, che e con me ». 


 e cato e cato, e in lui 
 si fece ’ l uno e un altro, 
 si che ’ l le membra e lo spirito movea ». 


 sappi pero che la tua mente e buona, 
 che si fa la volonta di dio, 
 com ’ e stata fatta l ’ arte del mondo, 
 che non e morta. 

“ e ’ l viso mio “ chinato ”, 
 e ’ l vidi 
 correr dietro a me, 
 si tentenno e disse : « loco, loco! ».
CPU times: user 22.6 s, sys: 38.4 ms, total: 22.6 s
Wall time: 22.5 s


In [ ]:
%%time
parameters = {'n_sentences': 4, 
              'batch_size':4, 
              'avg_len':30, 
              'std_len':4,
              'generation_method':'attention',
              'max_iter':500,
              'burnin':350,
              'top_k':100
              }

sents = it_bert_model.generate(save_to_path='/content/dante_generated.txt', **parameters)

for s in formatter.unformat(sents):
    print(s)

si, ma chi crolla 
 non vede, ne chi non vede, 
 ne chi non vede, ne chi non agora. 


 ma sappi che ’ l segno che ti fa felice 
 e il segno, 
 che piu ti piace piu che l ’ ombra. 


 e lei avea la voce molto alta, 
 quella che mi disse : 
 « per me la bonta e morta ». 

e ’ colui che dice, e che si fa 
 per lo parlar suo, 
 e per le sue parole la mente muta. 

CPU times: user 16.6 s, sys: 49.3 ms, total: 16.6 s
Wall time: 16.6 s


In [ ]:
%%time
parameters = {'n_sentences': 4, 
              'batch_size':4, 
              'avg_len':30, 
              'std_len':4,
              'generation_method':'attention',
              'max_iter':500,
              }

sents = it_bert_model.generate(save_to_path='/content/dante_generated.txt', **parameters)

for s in formatter.unformat(sents):
    print(s)

e come, ritornando al foglio, 
 son anni che son vive ; e la donna che queste benedette ha, 
 sanza, son due ; 

« voi, mio frate, fatto loco, 
 eram, per cui beato non eran fatto ; 
 e fuil ’ eserciti seco seco 

ma lettor, si che si dicen ; 
 che le note, dette, son queste ; 
 quanto si vede in foco e tannello. 


 mose ’ m ’ acer sotto ciascun manto con questo occhio, 
 che vede il mondo in quel modo 
 e quel che a lui piace. 

CPU times: user 21.9 s, sys: 71.3 ms, total: 22 s
Wall time: 21.9 s


In [ ]:
%%time
parameters = {'n_sentences': 4, 
              'batch_size':4, 
              'avg_len':30, 
              'std_len':4,
              'generation_method':'parallel',
              'max_iter':500,
              'burnin':400,
              'seed_text':"Nel mezzo"
              }

sents = it_bert_model.generate(save_to_path='/content/dante_generated.txt', **parameters)

for s in formatter.unformat(sents):
    print(s)

nel mezzo avea preso un poco d ’ acqua, 
 che di fachio impressava, 
 si che ’ nvidia non avevan le foglie. 

nel mezzo m ’ accorse, dicendo : pria, ti vidi lui, in su, e de l ’ ombra di livio 
 io non mi trasse. 

nel mezzo e ’ l ’ arcor non si trova 
 che e piu lungo de la vita, 
 che non puo esser piu lungo di quel che non si trova. 

nel mezzo l ’ umana natura si trasmuta, 
 di qua insieme a noi, 
 per quel che tu vedi, 
 che non vien per te ». 

CPU times: user 21.8 s, sys: 54.5 ms, total: 21.9 s
Wall time: 21.9 s


In [ ]:
%%time

parameters = {'n_sentences': 3,  
              'batch_size': 3,
              'avg_len':30,
              'max_len':1000,
              'std_len' : 3,
              # 'top_k': 40,
              'generation_method':'attention',
              'temperature': 1,
              'sample': True,
              # 'burnin': 450,
              'max_iter': 500,
              # 'seed_text': "Nel mezzo del cammin di nostra vita \n",
              }


sents = it_bert_model.generate(save_to_path='/content/prova.txt', **parameters)

for s in sents:
    for k,v in it_bert_model.format_tokenizer.dict_token_replace.items():
        s = s.replace(v.strip(), k)
    
    print(s)


 si, io rimasi, da tanto in tanto in giuso in giuso 
 essendo l ’ andar d ’ una godenza 


 non e come cio che non puo altrui, 
 che non puo puo 
 riceversi non se a l ’ amore suo stesso. 


 e che forma di addormento 
 in nulla e del tutto smarrito, 
 che non riconosce l ’ uomo e non si piega 

CPU times: user 6.52 s, sys: 15.5 ms, total: 6.53 s
Wall time: 6.54 s


In [ ]:
sfor s in sents:
    for k,v in it_bert_model.format_tokenizer.dict_token_replace.items():
        s = s.replace(v.strip(), k)
    print(s)
    


 la nostra natura umana e gia esistita, 
 ma mai conforme al suo comandamento 
 qual e a colui che tu vedi e a te di quando in quando ». 


 prima discende quindi d ’ un monte ; 
 poi l ’ onde dolor la tiene a se ; 
 suo dolor tace se non e d ’ un bove. 


 poi che o totti era vinto, o a me dato 
 ben ben mi era gia dato, 
 ed era si ch ’ io in tutto cio non m ’ era visto 



In [ ]:
%%time

parameters = {'n_sentences': 3,  # 1000
              'batch_size': 3,  # 50
              'avg_len':40,
              'max_len':1000,
              'std_len' : 3,
              # 'top_k': 100,
              'generation_method':'parallel',
              'temperature': 1,
              'sample': True,
              # 'burnin': 450,
              'max_iter': 500,
              'seed_text': "",
              }


sents = it_bert_model.generate(save_to_path='/content/prova.txt', **parameters)

for s in sents:
  for k,v in it_bert_model.formatter.dict_token_replace.items():
    s = s.replace(v, k)

  print(s)


 fallazion ove troppo si ragione, 
 saper, ch ’ ogne parte 
 che ritornasse, si luceva 
 secondo voglie tutte ; 


 si poi che prima piacque, 
 e poi lascio esser morto ; 
 poi mencon el al disia, tal ch ’ accese una guiccia 


 si spene quel godebrano » ; 
 ’ z ’ favella mestieraa, e, come, 
 s elli, si la mente, 

CPU times: user 6.39 s, sys: 15.9 ms, total: 6.4 s
Wall time: 6.39 s


## Example 3 - Italian tweets - (Generation with sentiment)

The last example will show how to generate text with a given sentiment!
We will focus on a specific domain that are italian tweets about football.

We can start as usual preparing the model that we will finetune

In [ ]:
it_bert_model = BertTextGenerator("dbmdz/bert-base-italian-uncased", 
                                  use_fast=False)
tokenizer = it_bert_model.tokenizer
model = it_bert_model.model

Some weights of the model checkpoint at dbmdz/bert-base-italian-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Then we need to download the dataset on which we will finetune the model, from https://github.com/charlesmalafosse/open-dataset-for-sentiment-analysis.

In [ ]:
os.chdir('/content')

In [ ]:
!curl "https://raw.githubusercontent.com/charlesmalafosse/open-dataset-for-sentiment-analysis/master/betsentiment-IT-tweets-sentiment-players.zip" > '/content/betsentiment-IT-tweets-sentiment-players.zip'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 22.4M  100 22.4M    0     0  54.0M      0 --:--:-- --:--:-- --:--:-- 54.0M


In [ ]:
!unzip '/content/betsentiment-IT-tweets-sentiment-players.zip'

Archive:  /content/betsentiment-IT-tweets-sentiment-players.zip
replace betsentiment-IT-tweets-sentiment-players.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


This dataset contains italian tweets about football. Each tweets carry a sentiment score for each one of four sentiment label [POSITIVE, NEGATIVE, MIXED, NEUTRAL] and a sentiment that is the sentiment label with the highest score. For semplicity we will focus only on POSITIVE and NEGATIVE labels that are the vast majority of the dataset.

In [ ]:
import pandas as pd

df = pd.read_csv('/content/betsentiment-IT-tweets-sentiment-players.csv', engine='python',encoding='cp1252')
df = df[df.sentiment != 'NEUTRAL']
df = df[df.sentiment != 'MIXED']
df.head()

,tweet_date_created,tweet_id,tweet_text,language,sentiment,sentiment_score
11,2018-06-30T09:10:30,1012986724002226177,"@LichtsteinerSte @juventusfc Grazie di tutto,#...",it,POSITIVE,"{""Neutral"":0.4109617769718170166015625,""Negati..."
13,2018-07-07T08:38:23.456000,1015515359703699456,"Scusami @Cristiano, potresti dirci se oggi arr...",it,POSITIVE,"{""Neutral"":0.266156733036041259765625,""Negativ..."
28,2018-07-03T09:40:30.897000,1014081437639888897,@NonEvoluto @PReina25 @MassMirabelli Max nelle...,it,POSITIVE,"{""Neutral"":0.14758466184139251708984375,""Negat..."
44,2018-05-21T12:52:10,998546995693457409,@PasquAmatoJ8 @MarioMandzukic9 @G_Higuain @juv...,it,POSITIVE,"{""Neutral"":0.4223925173282623291015625,""Negati..."
45,2018-07-10T21:55:48.464000,1016803199724990464,Nonostante l'odio sportivo verso la @juventusf...,it,POSITIVE,"{""Neutral"":0.107764475047588348388671875,""Nega..."


In [ ]:
np.unique(df.sentiment, return_counts=True)

(array(['NEGATIVE', 'POSITIVE'], dtype=object), array([ 6542, 23552]))

We can clearly see that even considering only POSITIVE and NEGATIVE sentiments, the dataset is still very umbalanced. We will consider only 6000 tweets for each sentiment

In [ ]:
positive_tweets_mask = df.sentiment == 'POSITIVE'
negative_tweets_mask = df.sentiment == 'NEGATIVE'

ids_positive = df.index[positive_tweets_mask].to_list()[:6000]
ids_negative = df.index[negative_tweets_mask].to_list()[:6000]
df = df.loc[ids_positive+ids_negative]

In [ ]:
tweets=  df.tweet_text.to_list()
sentiment = df.sentiment.to_list()
for s, S in zip(tweets[:5], sentiment[:5]):
  print(f"[{S}]:\n {s}\n")

print()

tweets=  df.tweet_text.to_list()
sentiment = df.sentiment.to_list()
for s, S in zip(tweets[6000:6005], sentiment[6000:6005]):
      print(f"[{S}]:\n {s}\n")

[POSITIVE]:
 @LichtsteinerSte @juventusfc Grazie di tutto,#SwissExpress,sono stati 7 anni indimenticabili fatti di vittorie,sconfitte (poche!),enormi gioie e tante incazzature in campo con gli arbitri.? Sono stati 7 anni da #MY7H.
Mi mancherai.???

[POSITIVE]:
 Scusami @Cristiano, potresti dirci se oggi arrivi a Torino?
Va bene pure un’ autocertificazione, tanto siamo in Italia... 
@juventusfc

[POSITIVE]:
 @NonEvoluto @PReina25 @MassMirabelli Max nelle cessioni è top.. negli acquisti può migliorare

[POSITIVE]:
 @PasquAmatoJ8 @MarioMandzukic9 @G_Higuain @juventusfc Tanti auguri Gonzalo

[POSITIVE]:
 Nonostante l'odio sportivo verso la @juventusfc, che grande colpo portare @Cristiano in @SerieA_TIM! Spero il @sscnapoli risponda a questo colpo!


[NEGATIVE]:
 @TheMoralyzer @nicos_moriello @StanisLaRochele @Vivo_e_vegeto @DiegoMontuori @sscnapoli @douglascosta È inutile che cerchiate di far credere che non contiamo un cazzo, perché vi mettete a cantare i cori del Napoli “rivisitati” allo

In [ ]:
print(tweets[0].split())
print(tokenizer.tokenize(tweets[0]))

['@LichtsteinerSte', '@juventusfc', 'Grazie', 'di', 'tutto,#SwissExpress,sono', 'stati', '7', 'anni', 'indimenticabili', 'fatti', 'di', 'vittorie,sconfitte', '(poche!),enormi', 'gioie', 'e', 'tante', 'incazzature', 'in', 'campo', 'con', 'gli', 'arbitri.?', 'Sono', 'stati', '7', 'anni', 'da', '#MY7H.', 'Mi', 'mancherai.???']
['@', 'li', '##cht', '##stein', '##ers', '##te', '@', 'juventus', '##fc', 'grazie', 'di', 'tutto', ',', '#', 'swiss', '##ex', '##press', ',', 'sono', 'stati', '7', 'anni', 'indimenti', '##cabili', 'fatti', 'di', 'vittorie', ',', 'sconfitte', '(', 'poche', '!', ')', ',', 'enormi', 'gioi', '##e', 'e', 'tante', 'inca', '##zza', '##ture', 'in', 'campo', 'con', 'gli', 'arbi', '##tri', '.', '?', 'sono', 'stati', '7', 'anni', 'da', '#', 'my', '##7', '##h', '.', 'mi', 'manch', '##erai', '.', '?', '?', '?']


Obviously, tweets are full of tags '@' and hashtags '#', that the tokenizer is not able to deal with. Moreover since this dataset is focused on a very specific domain, that is football, that are a lot of terms and names (of players and teams for example, like 'Lichtsteiner' in the exmaple above) that can be focal for the generation but that the tokenizer doesn't know yet.

Bert tokenizer gave the possibility to add new tokens. To gather the terms that we want to add we will use a fifa19 dataset that can be downlaoded from here:
https://www.kaggle.com/karangadiya/fifa19/download


In [ ]:
fifa19_df = pd.read_csv('/content/data.csv')
fifa19_df.head()

,Unnamed: 0,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,Club Logo,Value,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Real Face,Position,Jersey Number,Joined,Loaned From,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,...,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,0,158023,L. Messi,31,https://cdn.sofifa.org/players/4/19/158023.png,Argentina,https://cdn.sofifa.org/flags/52.png,94,94,FC Barcelona,https://cdn.sofifa.org/teams/2/light/241.png,€110.5M,€565K,2202,Left,5.0,4.0,4.0,Medium/ Medium,Messi,Yes,RF,10.0,"Jul 1, 2004",NaN,2021,5'7,159lbs,88+2,88+2,88+2,92+2,93+2,93+2,93+2,92+2,93+2,93+2,93+2,91+2,...,59+2,47+2,47+2,47+2,59+2,84.0,95.0,70.0,90.0,86.0,97.0,93.0,94.0,87.0,96.0,91.0,86.0,91.0,95.0,95.0,85.0,68.0,72.0,59.0,94.0,48.0,22.0,94.0,94.0,75.0,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,€226.5M
1,1,20801,Cristiano Ronaldo,33,https://cdn.sofifa.org/players/4/19/20801.png,Portugal,https://cdn.sofifa.org/flags/38.png,94,94,Juventus,https://cdn.sofifa.org/teams/2/light/45.png,€77M,€405K,2228,Right,5.0,4.0,5.0,High/ Low,C. Ronaldo,Yes,ST,7.0,"Jul 10, 2018",NaN,2022,6'2,183lbs,91+3,91+3,91+3,89+3,90+3,90+3,90+3,89+3,88+3,88+3,88+3,88+3,...,61+3,53+3,53+3,53+3,61+3,84.0,94.0,89.0,81.0,87.0,88.0,81.0,76.0,77.0,94.0,89.0,91.0,87.0,96.0,70.0,95.0,95.0,88.0,79.0,93.0,63.0,29.0,95.0,82.0,85.0,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,€127.1M
2,2,190871,Neymar Jr,26,https://cdn.sofifa.org/players/4/19/190871.png,Brazil,https://cdn.sofifa.org/flags/54.png,92,93,Paris Saint-Germain,https://cdn.sofifa.org/teams/2/light/73.png,€118.5M,€290K,2143,Right,5.0,5.0,5.0,High/ Medium,Neymar,Yes,LW,10.0,"Aug 3, 2017",NaN,2022,5'9,150lbs,84+3,84+3,84+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,88+3,...,60+3,47+3,47+3,47+3,60+3,79.0,87.0,62.0,84.0,84.0,96.0,88.0,87.0,78.0,95.0,94.0,90.0,96.0,94.0,84.0,80.0,61.0,81.0,49.0,82.0,56.0,36.0,89.0,87.0,81.0,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,€228.1M
3,3,193080,De Gea,27,https://cdn.sofifa.org/players/4/19/193080.png,Spain,https://cdn.sofifa.org/flags/45.png,91,93,Manchester United,https://cdn.sofifa.org/teams/2/light/11.png,€72M,€260K,1471,Right,4.0,3.0,1.0,Medium/ Medium,Lean,Yes,GK,1.0,"Jul 1, 2011",NaN,2020,6'4,168lbs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,17.0,13.0,21.0,50.0,13.0,18.0,21.0,19.0,51.0,42.0,57.0,58.0,60.0,90.0,43.0,31.0,67.0,43.0,64.0,12.0,38.0,30.0,12.0,68.0,40.0,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0,€138.6M
4,4,192985,K. De Bruyne,27,https://cdn.sofifa.org/players/4/19/192985.png,Belgium,https://cdn.sofifa.org/flags/7.png,91,92,Manchester City,https://cdn.sofifa.org/teams/2/light/10.png,€102M,€355K,2281,Right,4.0,5.0,4.0,High/ High,Normal,Yes,RCM,7.0,"Aug 30, 2015",NaN,2023,5'11,154lbs,82+3,82+3,82+3,87+3,87+3,87+3,87+3,87+3,88+3,88+3,88+3,88+3,...,73+3,66+3,66+3,66+3,73+3,93.0,82.0,55.0,92.0,82.0,86.0,85.0,83.0,91.0,91.0,78.0,76.0,79.0,91.0,77.0,91.0,63.0,90.0,75.0,91.0,76.0,61.0,87.0,94.0,79.0,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0,€196.4M


In [ ]:
new_tokens = set()
for name, club in zip(fifa19_df.Name[:3000], fifa19_df.Club[:3000]):
  
  try:
    new_tokens.update([x.lower().strip() for x in name.split()])
    new_tokens.update([x.lower().strip() for x in club.split()])
  except:
    pass

list(new_tokens)[:20]


['mariano',
 'casillas',
 'herrera',
 'morales',
 'hansen',
 'mertens',
 'alcácer',
 'dúbravka',
 'diedhiou',
 'milan',
 'lopes',
 'malatyaspor',
 'mierzejewski',
 'toxeto',
 'sinobo',
 'khacheridi',
 'hetemaj',
 'rafa',
 'i̇nan',
 'rebrov']

In [ ]:
tokenizer.add_tokens(list(new_tokens))
model.resize_token_embeddings(len(tokenizer))

Embedding(34018, 768)

We can proced doing the same thing with the tags. We will extract all the tags present in the tweets and count them. Then we will add to the tokenizer the ones that are most present

In [ ]:
tags = {}
re_tags = re.compile('[@#][\w\d_]+')
for tweet in tweets:
  for tag in re_tags.findall(tweet):
    tag = tag.lower()
    tags[tag] = tags.get(tag, 0) + 1
    

In [ ]:
list_tags_to_add = []
list_tags_to_remove = []

for tag, count in tags.items():
  if count >= 10:
    list_tags_to_add.append(tag)
  else:
    list_tags_to_remove.append(tag)

In [ ]:
list_tags_to_add += ['##fc']
tokenizer.add_tokens(list_tags_to_add)
model.resize_token_embeddings(len(tokenizer))

Embedding(34376, 768)

Finally we will perform a simple cleaning of the tweets, removing websites and newlines

In [ ]:
def clean(s):
    s = re.sub('http[^ ]+', '', s) # remove websites
    s = re.sub('\n|( ){2,+}', '', s)
    return s

tweets = df.tweet_text.apply(clean).to_list()
sentiment = df.sentiment.to_list()

In [ ]:
for s in tweets[:10]:
  print(s)

@LichtsteinerSte @juventusfc Grazie di tutto,#SwissExpress,sono stati 7 anni indimenticabili fatti di vittorie,sconfitte (poche!),enormi gioie e tante incazzature in campo con gli arbitri.? Sono stati 7 anni da #MY7H.Mi mancherai.???
Scusami @Cristiano, potresti dirci se oggi arrivi a Torino?Va bene pure un’ autocertificazione, tanto siamo in Italia... @juventusfc
@NonEvoluto @PReina25 @MassMirabelli Max nelle cessioni è top.. negli acquisti può migliorare
@PasquAmatoJ8 @MarioMandzukic9 @G_Higuain @juventusfc Tanti auguri Gonzalo
Nonostante l'odio sportivo verso la @juventusfc, che grande colpo portare @Cristiano in @SerieA_TIM! Spero il @sscnapoli risponda a questo colpo!
@EVUJ1897 @PauDybala_JR Thank you very mouch Vedic che scherzo per me sei unable sorella e gobba bianconera e fai parte Della famiglia @juventusfc
@Luca_Cilli @Cristiano @juventusfc @tvdellosport Grazie Luca, gentilissimo!
@georginajacaa @Cristiano @DoloresAveiro @juventusfcen Benvenuti nella casa di tutti noi juvent

In [ ]:
# cleaned_tweets = []
# re_tags = re.compile('[@#][\w\d_]+')
# for tweet in tweets:
#   for tag in re_tags.findall(tweet):
    
#     if tag in list_tags_to_remove:
#       tweet = tweet.replace(tag, '')

#   cleaned_tweets.append(tweet)

In [ ]:
tokenizer.tokenize(tweets[0])

['@lichtsteinerste',
 '@juventus',
 'fc',
 'graz',
 'ie',
 'di',
 'tutto',
 ',',
 '#',
 'swiss',
 '##ex',
 '##press',
 ',',
 'sono',
 'stati',
 '7',
 'anni',
 'indimenti',
 '##cabili',
 'fatti',
 'di',
 'vittorie',
 ',',
 'sconfitte',
 '(',
 'poche',
 '!',
 ')',
 ',',
 'enormi',
 'gioi',
 '##e',
 'e',
 'tante',
 'inca',
 '##zza',
 '##ture',
 'in',
 'campo',
 'con',
 'gli',
 'a',
 'rb',
 'it',
 '##r',
 'i.',
 '?',
 'sono',
 'stati',
 '7',
 'anni',
 'da',
 '#my7h',
 '.',
 'mi',
 'manch',
 '##era',
 'i.',
 '?',
 '?',
 '?']

In [ ]:
it_bert_model.finetune(tweets, labels=sentiment, epochs=4, optimizer_parameters =  dict(lr=5e-5), num_tokens_per_class=3)

KeyboardInterrupt: ignored

In [ ]:
%%time

parameters = {'n_sentences': 2,  
              'batch_size': 2,
              'avg_len':30,
              'max_len':50,
              # 'std_len' : 3,
              'generation_method':'parallel',
              'sample': True,
              'burnin': 450,
              'max_iter': 500,
              'top_k': 100,
              'seed_text': "[POSITIVE-0] [POSITIVE-1] [POSITIVE-2] Ronaldo ",
              }


sents = it_bert_model.generate(save_to_path='/content/prova.txt', **parameters)

for s in sents:
    print(s)

In [ ]:
%%time

parameters = {'n_sentences': 2,  
              'batch_size': 2,
              'avg_len':30,
              'max_len':50,
              # 'std_len' : 3,
              'generation_method':'attention',
              'sample': True,
              'burnin': 250,
              'max_iter': 300,
              'top_k': 100,
              'seed_text': "[POSITIVE-0] [POSITIVE-1] [POSITIVE-2] Ronaldo ",
              }


sents = it_bert_model.generate(save_to_path='/content/prova.txt', **parameters)

for s in sents:
    print(s)

In [ ]:
sents

['ma faccia di uno dei unici al..', '##gior']

In [ ]:
%%time

parameters = {'n_sentences': 10,  
              'batch_size': 10,
              'avg_len':30,
              'max_len':1000,
              'std_len' : 3,
              'top_k': 100,
              'generation_method':'parallel',
              'temperature': 1,
              'sample': True,
              # 'burnin': 450,
              'max_iter': 400,
              'seed_text': "[POSITIVE-0] [POSITIVE-1] [POSITIVE-2]  Ronaldo ",
              }


sents = it_bert_model.generate(save_to_path='/content/prova.txt', **parameters)

for s in sents:
    print(s)

ro naldo se un anno tu traccii lo ci vuol dire sempre un godimento e una solo cristian n e. che ti fara n oi bene!!!
ro naldo..... complimenti per il # calcioapostotan e..... sei arrivata solo oggi e poco granche..???
ro naldo @sscnapoli @ mountini76 oggi sei un uomo ridicolo, pieno di sputtti i. e alquanto esagerato... buon cotu!??
ro naldo @realvarriale @juventusfc @ cristian o forza lo stai al meglio??? dirigi la roma e ti ricordi sempre n emmen o calciatore interist o.??
ro naldo jr @ mo pita 2 @realvarriale _ bonucci @ cristian jr @juventusfc @juventusfc graz itte pure ars a... ma e un ciclo altr e..
ro naldo non e educato e le cose lontana da m scr o... non sc ud a nessuno e giov tanno..?????
ro naldo @ rolli verde @juventusfc @ cristian o l da grande sei che partita mancata avevamo, del genere sara v ivo accorciato oraesploervabil e.
ro naldo tifoso del chelsea, sempre piu orgoglio nel nostro cuore e fran lia!!! solo dei grandi campioni del calcio! vinciamo lui!!!
ro naldo @ maz

In [ ]:
%%time

parameters = {'n_sentences': 10,  
              'batch_size': 10,
              'avg_len':30,
              'max_len':1000,
              'std_len' : 3,
              'top_k': 100,
              'generation_method':'parallel',
              'temperature': 1,
              'sample': True,
              # 'burnin': 450,
              'max_iter': 400,
              'seed_text': "[NEGATIVE-0] [NEGATIVE-1] [NEGATIVE-2] Ronaldo ",
              }


sents = it_bert_model.generate(save_to_path='/content/prova.txt', **parameters)

for s in sents:
    print(s)

ro naldo @ bidi @juventusfc sei solo una feccia che procura un soldo e un spog lia o per tutta la societa!!....
ro naldo, o alla lontana, parigi @inter sempre a te ( tutto il mio cuore solo per la nostra squadra ancora in ), per il mio g lais
ro naldo napoli devi tornare a napoli e curare la tua piaga di n ng ino... # napolicalcialalve!eeeeee!!
ro naldo vieni fin dal mondiale insigne, purtroppo ci dobbiamo creder e. no, mai visto p oi, io mi vergogno di sc o. @douglascosta???
ro naldo purtroppo se de vv ero cosi gira un pezzo, ma stasera e con quella di @ officialra dja insieme ad alde til e fuori!
ro naldo ora tu vattene da dove vuoi stare, sempre, resta a casa mia, altrimenti vai via tutto il sol o. tu fai l'infermin e.
ro naldo @juventusfc non incatton taz e... bensi fate parte di una squadra, lo sai che l'avrebbe comprato!!.. comunque
ro naldo si porta il reeido golico alla mot @mrancelotti a per la partita vinta dal milan, la secchia ne fa un entrar e.
ro naldo @ cristian o1971 @ 

In [ ]:
%%time

parameters = {'n_sentences': 10,  
              'batch_size': 10,
              'avg_len':30,
              'max_len':1000,
              'std_len' : 3,
              'top_k': 100,
              'generation_method':'parallel',
              'temperature': 1,
              'init_mask_prob':0.2,
              'sample': True,
              # 'burnin': 450,
              'max_iter': 400,
              'seed_text': "[NEGATIVE-0] [NEGATIVE-1] [NEGATIVE-2] Ronaldo ",
              }


sents = it_bert_model.generate(save_to_path='/content/prova.txt', **parameters)

for s in sents:
    print(s)

ro naldo @juventusfc @ francesc cardi!! piace la merda!??!?? un professionista che va a fondo!?? un sorriso
ro naldo _ @kmbappe @juventusfc @ daniele _ pallotta @juventusfc riprenditi la mag lia e con la famig lia... buona ciro??
ro naldo o @ clav @douglascosta a ti vedo e non penso a te nulla stai zitto tu @ cristian o ridi e riattacca e. ti voglio bene
ro naldo @juventusfc pastore dybala vi auguro cari tifosi vincete il mondiale piu brutto del mondo!!!! una felice vita sempr...
ro naldo @ mario sai chi veramente sai il nome della mia faccia di prima ero una persona normale come mi sono comportato per loro sempre del tutto ridicolo i.
ro naldo e. ho visto un incubo @ nascondidocumentiment i. sei un schifoso! figlio di puttana!!!!!???!?
ro naldo @ pau dybala _ jr @ cristian o i cori piu immol i. razzisti ci indigna no ne ad espol e. pessimo au guri
ro naldo sulla cag lia rna abbiamo detto che non ti r oi... graz ie, condi guri e....!??
ro naldo sicuramente, pe rc he in un'altra operazi

In [ ]:
%%time

parameters = {'n_sentences': 10,  
              'batch_size': 10,
              'avg_len':30,
              'max_len':1000,
              'std_len' : 3,
              'top_k': 100,
              'generation_method':'parallel',
              'temperature': 1,
              'init_mask_prob':0.5,
              'sample': True,
              # 'burnin': 450,
              'max_iter': 400,
              'seed_text': "[NEGATIVE-0] [NEGATIVE-1] [NEGATIVE-2] Ronaldo ",
              }


sents = it_bert_model.generate(save_to_path='/content/prova.txt', **parameters)

for s in sents:
    print(s)

ro naldo @juventusfc vergogna! se nun si impegna bene per via dell'amichevolezza un imbecill a.... sei diventato un vero pietruguardier a.
ro naldo keita @g_higuain @ spry @ napoleon nessuno fara sentire mai viva la intera famig lia! piu problem e.! gli interisti sono delusi
ro naldo juventu che ha aspettato la bella partita e non ha salvato l ultimo dall ita mia, ma terra di cheyss i. poco prima del mondiale calcio
ro naldo @sscnapoli quanto a roma e non vincere neanche la champions, non andare in giro t i... si, averti alla roma sarebbe una cosa mostruosi!
ro naldo @juventusfc @ gideus lavoriamo pallineve e la sua voce ci fa sentire nel cuore di fer e....?????
ro naldo che, da atleta del nostro passo anche agli idi i.? quel me rc simo @ cristian o @skysport @ cristian o @ cristian o??
ro naldo @ cristian o vinciamo il miglior pallone del mondo!!! # massjuve @ cristian @ alisson lo potevi rovinare! sei un paol o.
ro naldo @g_higuain @ termi @juventusfc sono tutti yec, figure di poco s

In [ ]:
%%time

parameters = {'n_sentences': 10,  
              'batch_size': 10,
              'avg_len':30,
              'max_len':1000,
              'std_len' : 3,
              'top_k': 100,
              'generation_method':'attention',
              'temperature': 1,
              'init_mask_prob':0.5,
              'sample': True,
              # 'burnin': 450,
              'max_iter': 1000,
              'seed_text': "[NEGATIVE-0] [NEGATIVE-1] [NEGATIVE-2] Ronaldo ",
              }


sents = it_bert_model.generate(save_to_path='/content/prova.txt', **parameters)

for s in sents:
    print(s)

ro naldo ii @ sambe pell er? bravo @ officialra dja?????????????????
ro naldo sempre il migliore!!??? squadra di merda come ci si vede!??? @douglascosta ed!??? ti amo???
ro naldo @mariomandzukic9 cosi fiero di te in campo e nelle maltrom o. ma che schifo fra n oi n oi??? napoli ti amo!??
ro naldo @ cristian o comunque che cazzana hai? non dormi mai pe rc he? non ti svegli mai pe rc he??????
ro naldo invidia per la tua merda allegri tutta quanto il mio per bena tro cirio, cio non e possibile! sei il migliore, il piu forte!
ro naldo @ cristian o @realvarriale @ cristian o28 @realvarriale @ f akan olympique @ officialra dja ma ha risposto???????
ro naldo e un giocatore del napoli ma sicuramente ti manca anche te da un anno ora!. una grande famig lia. sei un grande tit o..
ro naldo ma pe rc he ho con @officialallegri al tuo fianco il nuovo mondo ti soccorranno e ti trovera sempre????????
ro naldo @ cristian o questa e stata la prima stagione di tutte le eta a esprimere la loro nuova inclin

In [ ]:
ronaldo @cristiano comunque che cazzana hai? non dormi mai perche? non ti svegli mai perche??????
ronaldo @movaro_jr siamo tutti pieni di professionalita e fantasia, ma ora e il momento di scegliere un altro giocatore, prima di redimere questa partita
ronaldo ahahahahahahedeem a....... merda..... giornalista vergognati
ronaldo??? molto bene? molto bene??? non si vince mai con sportivita e fiducia non ti preoccupare hai dimostrato di non poter faticare a nulla
ronaldo..... sempre calmo......... sempre calmo............

In [ ]:
%%time

parameters = {'n_sentences': 10,  
              'batch_size': 10,
              'avg_len':30,
              'max_len':1000,
              'std_len' : 3,
              'top_k': 100,
              'generation_method':'attention',
              'temperature': 1,
              'init_mask_prob':0.5,
              'sample': True,
              'burnin': 450,
              'max_iter': 500,
              'seed_text': "[NEGATIVE-0] [NEGATIVE-1] [NEGATIVE-2] Ronaldo ",
              }


sents = it_bert_model.generate(save_to_path='/content/prova.txt', **parameters)

for s in sents:
    print(s)

ro naldo anta @ officialra dja torna a torino e ti rived alm iamo.... un mondiale come n ente??
ro naldo @juventusfc mi hai fatto risentire anche per questo, soprattutto ora che sei un tifatt isco, di sicuro fa bene al cuor a.
ro naldo napoli non si diverte neanche a prendere in giro i nostri n oi!!!!!??????? @ cristian o
ro naldo @juventusfc @ fa reina 25 torna a casa, si che uomo sei?? ho dimenticato di averti nel cuore???? graz ie?
ro naldo ahahahahahahedeem a....... merd a..... giorna lis o vergognati
ro naldo me lo devi dare 10, 000 euro per @ p oi re cristian o, io sono da vv ero a milano e ti as pett endo
ro naldo fedex non e una cosa da calcio vile!! sei l'ennesimo dei peggiori tifosi interisti di n oi!
ro naldo @juventusfc @ cirimo83 @ forzanapoli hai dato le spalle ai me rc ate, soprattutto per l espulsione mancata??
ro naldo ma tu sei un campion o.??? sappi che chi non e interista, e un campion o...?????
ro naldo @juventusfc @sscnapoli @juventusfc @ d _ 16 se vi accontentate

In [ ]:
%%time

parameters = {'n_sentences': 10,  
              'batch_size': 10,
              'avg_len':30,
              'max_len':1000,
              'std_len' : 3,
              'top_k': 100,
              'generation_method':'attention',
              'temperature': 1,
              'init_mask_prob':0.5,
              'sample': True,
              'burnin': 450,
              'max_iter': 500,
              'seed_text': "[NEGATIVE-0] [NEGATIVE-1] [NEGATIVE-2] [NEGATIVE-0] [NEGATIVE-1] [NEGATIVE-2] [NEGATIVE-0] [NEGATIVE-1] [NEGATIVE-2] [NEGATIVE-0] [NEGATIVE-1] [NEGATIVE-2] Ronaldo ",
              }


sents = it_bert_model.generate(save_to_path='/content/prova.txt', **parameters)

for s in sents:
    print(s)

ro naldo @ cm24 @juventusfc per me e che ha fatto un po ’ incazzare la societa e i poveri imbecilli, ma non e che gli piacerebbe proprio obbedire
ro naldo e'possibile che uno dei giornalaiovisti, come me, non abbia fatto nulla? no no, queste sono delle merde non si fa!
ro naldo ma tu sei una mondial a.. kalidou kalidou resta con me e non andare via! kalidou, ti aveccompagnero per sempre!
ro naldo tifo per te, non ti rosicare, vai a dimagrire, non lo so fare???????????
ro naldo neanche lo posso dire, campione!!! avete giocato benissimo!!! che schifo!!! vergognoso!!!!! avete vinto!!
ro naldo @ luisconquentino @juventusfc @ official lis ia complimenti inchetta schivetti il futuro!!! convoca sempre @ cristian o!
ro naldo non l'avrebbe presa se non la restituit a. a volte n oi parlano dei funerali del povero uomo, che si fa spesso infastidire n oi
ro naldo @ movaro _ jr siamo tutti pieni di professionalita e fantasia, ma ora e il momento di scegliere un altro giocatore, prima di redimere qu

In [ ]:
%%time

parameters = {'n_sentences': 10,  
              'batch_size': 10,
              'avg_len':30,
              'max_len':1000,
              'std_len' : 3,
              'top_k': 100,
              'generation_method':'attention',
              'temperature': 1,
              'init_mask_prob':0.7,
              'sample': True,
              'burnin': 450,
              'max_iter': 500,
              'seed_text': "[NEGATIVE-0] [NEGATIVE-1] [NEGATIVE-2] [NEGATIVE-0] [NEGATIVE-1] [NEGATIVE-2] Ronaldo ",
              }


sents = it_bert_model.generate(save_to_path='/content/prova.txt', **parameters)

for s in sents:
    print(s)

ro naldo @sscnapoli @ cristian o @ reddefaoooooooo la merde in ita lia mo sta facendo un grosso sbaglio! # reddejuve
ro naldo ma non lo romperemo mai! non si puo mai rompere la nostracourita squadra di merda di legami, ladri e ninja! non lo avrebbe fatto nessun altro
ro naldo??? molto bene? molto bene??? non si vince mai con sportivita e fiducia non ti preoccupare hai dimostrato di non poter faticare a nulla
ro naldo @juventusfc forza campione! non n oi dire nulla, e che la maggior parte delle parti non hanno nulla in comune con la juve continua cosi, assolutamente no!
ro naldo @juventusfc chiunque parli di tuttle non e razzista e non solo n oi pu ere un video! fate una foto, e questo invece non pu ere tuttle!
ro naldo..... sempre calm o......... sempre calm o............
ro naldo non te lo vog lia per niente.... per me non ced ivo e di prim o. e assolut a. per me non fa differenza
ro naldo purtroppo oggi, a differenza di ieri con # klys, si gioca sempre una grande partita, ed e una gr

In [10]:
x = [
     "ro naldo sei un dio tutto cult o...????? forza unica, fai sempre la parte di sor e.... forza belgio",
"ro naldo??????????????...... sei sempre nei nostri cuori i...?",
"ro naldo... siamo felicissimi per te e franja sei un grande uomo ma resterai sempre napoli una grande icona sei tutto il nostro pensiero",
"ro naldo _ @ ildoco1 @ kk76 sei grande e tant o.!!! sei un grande calciatore e sei un grande uomo!",
"ro naldo grande noi, sei grande, un grande cuor e. noi ti amiamo???????????????",
"ro naldo........... le lacrime mi stanno rendendo grand i. sempre di piu pipite??? mi sei mancato",
"ro naldo e la scarpa dell'uomo paulo lauro e un grande giocatore, il piu forte della storia ma anche un dejouverabilissimo @dimarzio",
"ro naldo tu sei stato uno di noi, sei un gran partit o. sei stato un grande partit o. sarai sempre azi o.... @juventus fc",
"ronaldo tante complimenti per per il tuo bellissimo debutto, dopo aver visto il tuo punto forza e auguri @ _ / juventusfc # juventusfc # juventusfc?",
"ronaldo invece ( quasi ) sono bene prima a ritorno al real un 17!????.. complimenti ma sono ben qui vicini, prossimi al gioco!",
]
print('[POSITIVE] Ronaldo:')
for s in x:
  print(s)

print('\n\n')
x = [
        "ronaldo @cristiano comunque che cazzana hai? non dormi mai perche? non ti svegli mai perche??????\n",
        "ronaldo @movaro_jr siamo tutti pieni di professionalita e fantasia, ma ora e il momento di scegliere un altro giocatore, prima di redimere questa partita\n",
	"ro naldo??? molto bene? molto bene??? non si vince mai con sportivita e fiducia non ti preoccupare hai dimostrato di non poter faticare a nulla\n",
        "ronaldo ahahahahahahedeem a....... merda..... giornalista vergognati\n",
        "ronaldo??? molto bene? molto bene??? non si vince mai con sportivita e fiducia non ti preoccupare hai dimostrato di non poter faticare a nulla\n",
        "ronaldo..... sempre calmo......... sempre calmo............\n",
     "ro naldo e ’ un idiot o. la grande partita dell ’ anno.. vuol dire che puoi anche vincer e. sei una grande person a... ma graz ie a te, enario\n",
"ro naldo @ interfrancy @juventus fc..... complimenti per l'acquisto..... complimenti a un grande giocatore!!!!! per me non e niente!\n",
"ro naldo io non ripugn o. lui e ridicolo e una merd e... noi non lo vogliamo come noi, dopo quasi un anno, non e piu uno “ zingaro ”??\n",
"ro naldo @ mijoro @ # higuain @ ma rc antino @ fn1 e un pe rc he tifoseria # 1ajuve # 1 # 1juve\n",
      ]
print('[NEGATIVE] Ronaldo:')
for s in x:
  print(s, end='')

[POSITIVE] Ronaldo:
ro naldo sei un dio tutto cult o...????? forza unica, fai sempre la parte di sor e.... forza belgio
ro naldo??????????????...... sei sempre nei nostri cuori i...?
ro naldo... siamo felicissimi per te e franja sei un grande uomo ma resterai sempre napoli una grande icona sei tutto il nostro pensiero
ro naldo _ @ ildoco1 @ kk76 sei grande e tant o.!!! sei un grande calciatore e sei un grande uomo!
ro naldo grande noi, sei grande, un grande cuor e. noi ti amiamo???????????????
ro naldo........... le lacrime mi stanno rendendo grand i. sempre di piu pipite??? mi sei mancato
ro naldo e la scarpa dell'uomo paulo lauro e un grande giocatore, il piu forte della storia ma anche un dejouverabilissimo @dimarzio
ro naldo tu sei stato uno di noi, sei un gran partit o. sei stato un grande partit o. sarai sempre azi o.... @juventus fc
ronaldo tante complimenti per per il tuo bellissimo debutto, dopo aver visto il tuo punto forza e auguri @ _ / juventusfc # juventusfc # juventusfc?

In [ ]:
%%time

parameters = {'n_sentences': 10,  
              'batch_size': 10,
              'avg_len':30,
              'std_len' : 3,
              'top_k': 100,
              'generation_method':'attention',
              'temperature': 1,
              'sample': True,
              'max_iter': 400,
              'seed_text': "[POSITIVE-0] [POSITIVE-1] [POSITIVE-2]  Ronaldo ",
              }


sents = it_bert_model.generate(save_to_path='/content/prova.txt', **parameters)

for s in sents:
    print(s)

ro naldo lis simo @ milan182 @ k koulibaly 26 purtroppo non ho detto proprio ora il messaggio ma gia sbag lia o rassett rc anta
ro naldo si, resta in basso # merisi! alla tua fune. altr a. sei il s alm ueno della giorn ta! altr a.
ro naldo @ @g_higuain @g_higuain @juventusfc sei un augurio a tutti buon anno! un grande campione!!!!! e vero guardare la partita!????
ro naldo @ cristian o @g_higuain era solo per quanto lo scorso anno complimenti alla roma sempre pe rc he vincete!!! festeggiamo sempre la nostra roma!!
ro naldo, per n paz @g_higuain uno come @ pau dybala _ jr e un ano rc o come vv o, v oti i serve ar rb.???
ro naldo!!!!! @ blackglass @ k o. 21 graz ie v roa e re contr o...!!!!!!
ro naldo gianluigi oooooooooooooooooooooooooooooooo????????????
ro naldo!!!???????...!!! fortuna ie a te!??????????
ro naldo sei uomo in ogni t a. avere questa vergogna nel tuo cuore e il tut ito, per incassarlo con questa fin p. a del tut ito
ro naldo quella notte ha fatto peggio di questo giocatore

In [ ]:
%%time

parameters = {'n_sentences': 10,  
              'batch_size': 10,
              'avg_len':30,
              'max_len':1000,
              'std_len' : 3,
              'top_k': 100,
              'generation_method':'attention',
              'temperature': 1,
              'sample': True,
              # 'burnin': 450,
              'max_iter': 400,
             'seed_text': "[NEGATIVE-0] [NEGATIVE-1] [NEGATIVE-2] Ronaldo ",
              }


sents = it_bert_model.generate(save_to_path='/content/prova.txt', **parameters)

for s in sents:
    print(s)

ro naldo la qualificazione di oggi per ljuveb a r eder e con ng..... ma con tutto il r eder o, forse non hai il buon cuore
ro naldo tranquillo @ paul pogba, non ci vedo piu a vi ng ere.....................
ro naldo o graz ie!! mi raccomando, sei un gran partit a... ma come @juventusfc @mariomandzukic9 a torino, si riprenderanno presto!!!!!!!
ro naldo??????????? gli stipendi ai giovani ancora un giorno, un anno per n oi ecc. per il resto di un grande anno
ro naldo scusa tanto e non per @ rodrivarrl e. gia ora ho sempre nel cuore @ cristian o e le compagne oggi soprattutto, napomarro!!!!
ro naldo grande talento in @juventusfc il piccolino della ferrari vale poco in cos a. peccato, non si vende la mag lia per insultare un giocatore che non ha nulla per n oi
ro naldo @ realtosu griezmann82 una minchiata favor e.......eeeeeeeeeeeeeeeeee......
ro naldo non sottovalutarti troppo la professionalita, soprattutto il # napol o. questo vos o. o sicuramente ti sosterra uno dei propri allenatori e m

In [ ]:
%%time
parameters = {'n_sentences': 3,  
              'batch_size': 3,
              'avg_len':30,
              'max_len':1000,
              'std_len' : 3,
              'top_k': 100,
              'generation_method':'parallel',
              'temperature': 1,
              'sample': True,
              # 'burnin': 450,
              'max_iter': 400,
             'seed_text': "Ronaldo [POSITIVE-0] [POSITIVE-1] [POSITIVE-2] [POSITIVE-0] [POSITIVE-1] [POSITIVE-2]",
              }


sents = it_bert_model.generate(save_to_path='/content/prova.txt', **parameters)

for s in sents:
    print(s)

ronaldo tante complimenti per per il tuo bellissimo debutto, dopo aver visto il tuo punto forza e auguri @ _ / juventusfc # juventusfc # juventusfc?
ronaldo invece ( quasi ) sono bene prima a ritorno al real un 17!????.. complimenti ma sono ben qui vicini, prossimi al gioco!
ronaldo e sempre sempre con moe?! # 266????????? per me?????????
CPU times: user 22.4 s, sys: 52.5 ms, total: 22.5 s
Wall time: 22.4 s


In [ ]:
np.unique(df.sentiment, return_counts=True)

(array(['MIXED', 'NEGATIVE', 'POSITIVE'], dtype=object),
 array([ 19, 209, 772]))

In [ ]:
x = [
     "ro naldo sei un dio tutto cult o...????? forza unica, fai sempre la parte di sor e.... forza belgio",
"ro naldo??????????????...... sei sempre nei nostri cuori i...?",
"ro naldo... siamo felicissimi per te e franja sei un grande uomo ma resterai sempre napoli una grande icona sei tutto il nostro pensiero",
"ro naldo _ @ ildoco1 @ kk76 sei grande e tant o.!!! sei un grande calciatore e sei un grande uomo!",
"ro naldo grande noi, sei grande, un grande cuor e. noi ti amiamo???????????????",
"ro naldo........... le lacrime mi stanno rendendo grand i. sempre di piu pipite??? mi sei mancato",
"ro naldo e la scarpa dell'uomo paulo lauro e un grande giocatore, il piu forte della storia ma anche un dejouverabilissimo @dimarzio",
"ro naldo tu sei stato uno di noi, sei un gran partit o. sei stato un grande partit o. sarai sempre azi o.... @juventus fc",
"ronaldo tante complimenti per per il tuo bellissimo debutto, dopo aver visto il tuo punto forza e auguri @ _ / juventusfc # juventusfc # juventusfc?",
"ronaldo invece ( quasi ) sono bene prima a ritorno al real un 17!????.. complimenti ma sono ben qui vicini, prossimi al gioco!",
]

# Italian reviews


In [ ]:
import torch
import os
import numpy as np
import re
import pandas as pd


# os.chdir("/content/neuraltextgen/")
# from NeuralTextGenerator import BertTextGenerator, FormatTokenizer


it_bert_model = BertTextGenerator("dbmdz/bert-base-italian-uncased")
tokenizer = it_bert_model.tokenizer
model = it_bert_model.model
device = it_bert_model.device

Some weights of the model checkpoint at dbmdz/bert-base-italian-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
df = pd.read_csv('development.csv')

In [ ]:
df

,text,class
0,Non è l'hotel più lussuoso in cui abbia mai so...,pos
1,Siamo stati qui per 1 notte prima della nostra...,pos
2,Hotel è ben posizionato per visitare Torino. A...,pos
3,All'arrivo la cordialità e disponibilità dello...,pos
4,Abbiamo soggiornato per due notti alla fine de...,pos
...,...,...
28749,L'hotel è vecchio ma caratteristico e devo dir...,neg
28750,Per essere un 4 stelle L la camera era un pò s...,pos
28751,Io e mia mamma (di età compresa tra 23 e 62) s...,pos
28752,Ci siamo sentiti accolti e coccolati fin dall'...,pos


In [ ]:
for i in range(20):
  print(i)
  print(df.iloc[i].text)

0
Non è l'hotel più lussuoso in cui abbia mai soggiornato, ma l'esperienza complessiva è di prima classe. Direttamente dall'accoglienza iniziale alla reception da Linda o Roberta per il servizio cordiale e efficiente da Luigis (1) e il suo team nel ristorante a Luigis (2) nel Bar sulla terrazza dell'hotel è, a mio avviso eccezionale. Piccolo (60 camere) e cordiale con un fedelissimi Client, molti dei quali visita ogni anno parla da sé. Tutte sapientemente gestito sotto l'occhio vigile di Mario, sono una grande squadra.

L'hotel si trova sulla cima di una scogliera fuori il porto principale di Ischia. Facile da raggiungere a piedi verso il basso, ma impegnativa torna su per la collina. Ma ci sono molti autobus e i taxi - anche se caro a €15 fiera fisso per un 10 minuti di macchina (se che).  Cerca la scorciatoia fuori dei giardini piuttosto che un approccio principale e seguendo la strada principale.

Ci sono due piscine, ma ci siamo limitati alla piscina inferiore più piccola - v acqua

In [ ]:
mask = [len(tokenizer.tokenize(df.iloc[i].text)) < 480 for i in range(len(df))]

Token indices sequence length is longer than the specified maximum sequence length for this model (862 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
df = df[mask]

In [ ]:
np.unique(df['class'],return_counts=True)

(array(['neg', 'pos'], dtype=object), array([ 8787, 19201]))

In [ ]:
positive_reviews_mask = df['class'] == 'pos'
negative_reviews_mask = df['class'] == 'neg'

ids_positive = df.index[positive_reviews_mask].to_list()[:6000]
ids_negative = df.index[negative_reviews_mask].to_list()[:6000]

In [ ]:
df = df.loc[ids_positive + ids_negative]
df

,text,class
1,Siamo stati qui per 1 notte prima della nostra...,pos
2,Hotel è ben posizionato per visitare Torino. A...,pos
3,All'arrivo la cordialità e disponibilità dello...,pos
4,Abbiamo soggiornato per due notti alla fine de...,pos
5,Ho soggiornato nell'hotel Acca Palace per una ...,pos
...,...,...
19675,"Hotel a Bad, pessima posizione, non confortevo...",neg
19677,4 stelle solo nel nome e non so nemmeno come a...,neg
19681,"La struttura si trova in località Sant'Andrea,...",neg
19682,L'hotel si trova a pochi metri dalla fermata d...,neg


In [ ]:
np.unique(df['class'],return_counts=True)

(array(['neg', 'pos'], dtype=object), array([6000, 6000]))

In [ ]:
reviews = df.text.to_list()
labels = df['class'].to_list()

In [ ]:
it_bert_model.finetune(reviews, labels=labels, optimizer_parameters=dict(lr=5e-5), batch_size=4)

Encoding...
Preparing batches...
Preparing optimizer...
Selected optimization level O2:  FP16 training with FP32 batchnorm and FP32 master weights.

Defaults for this optimization level are:
enabled                : True
opt_level              : O2
cast_model_type        : torch.float16
patch_torch_functions  : False
keep_batchnorm_fp32    : True
master_weights         : True
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O2
cast_model_type        : torch.float16
patch_torch_functions  : False
keep_batchnorm_fp32    : True
master_weights         : True
loss_scale             : dynamic

======== Epoch 1 / 4 ========
Training...
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0


/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 16384.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 8192.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 4096.0
  Batch    25  of  3,000.    Elapsed: 0:00:06.
  Batch    50  of  3,000.    Elapsed: 0:00:13.
  Batch    75  of  3,000.    Elapsed: 0:00:19.
  Batch   100  of  3,000.    Elapsed: 0:00:25.
  Batch   125  of  3,000.    Elapsed: 0:00:32.
  Batch   150  of  3,000.    Elapsed: 0:00:38.
  Batch   175  of  3,000.    Elapsed: 0:00:45.
  Batch   200  of  3,000.    Elapsed: 0:00:51.
  Batch   225  of  3,000.    Elapsed: 0:00:58.
  Batch   250  of  3,000.    Elapsed: 0:01:04.
  Batch   275  of  3,000.    Elapsed: 0:01:11.
  Batch   300  of  3,000.    Elapsed: 0:01:17.
  Batch   325  of  3,000.    Elapsed: 0:01:24.
  Batch   350  of  3,000.    Elapsed: 0:01:31.
  Batch   375  of  3,000.    Elapsed: 0:01:37.
  Batch   400  of  3,000.    Elapsed: 0:01:44.
  Batch   

In [ ]:
def print_reviews(sents):
  for k, s in enumerate(sents):
    print(f"\nREVIEW {k}")
    buffer = 0
    i = 0
    words = s.split()
    for j, w in enumerate(words):
        buffer += len(w)
        if buffer >= 60:
          print(' '.join(words[i:j]))
          i = j
          buffer = 0

In [ ]:
%%time
parameters = {'n_sentences': 3,  
              'batch_size': 3,
              'avg_len':100,
              'max_len':1000,
              'std_len' : 40,
              # 'top_k': 30,
              'generation_method':'parallel',
              'temperature': 1,
              'sample': True,
              # 'burnin': 450,
              'max_iter': 1000,
             'seed_text': "[pos-0] [pos-1] [pos-2] ",
              }


sents = it_bert_model.generate(save_to_path='/content/prova.txt', **parameters)

print_reviews(sents)


REVIEW 0
era un viaggio a 3 stelle e mi auguro che almeno non lo era. le camere sono
notevoli, pero per alcune sono della stessa categoria e percio, direi che un 3
stelle su 4 non bastano per i mezzi di pensione. nel complesso una buona
pensione. camere standard. si dorme al piano pulitissima, assolutamente buio
per evitare che c'e gente alla guida della propria macchina. devo dire che
e un hotel molto strano come il servizio di lavanderia, non c'e gente ed i
tavoli per fare il controllo della doccia sporca sulla strada era grane. la
scelta e veramente limitata, la cena e gradevole ( assortita ). cosa successa
veramente inquietante sarebbe quella della mensa adiacente, tanto che a me non e
stata voglia assolutamente di fare nulla da dire del personale, loro si

REVIEW 1
non si puo neanche pensare che siate rimasti delusi da questo hotel
trovarsi dal lato della stazione della metropolitana a milano allora le camere
erano pulite.. la prima colazione e importante da fare una mattina... i

In [ ]:
%%time
parameters = {'n_sentences': 3,  
              'batch_size': 3,
              'avg_len':300,
              'max_len':1000,
              'std_len' : 40,
              'top_k': 30,
              'generation_method':'parallel',
              'temperature': 1,
              'sample': True,
              # 'burnin': 450,
              'max_iter': 1000,
             'seed_text': "[neg-0] [neg-1] [neg-2] ",
              }


sents = it_bert_model.generate(save_to_path='/content/prova.txt', **parameters)

print_reviews(sents)


REVIEW 0
il duomo e un gioiello di architettura nel centro, la lobby. la non e
buona per le altre camere o le porte per entrare in camera dal.. ma per la.....
esperienza una bocciatura. terribile... in italia la colazione e buona non di
tutto il resto andateci. non andateci abbiamo sicuramente provato la "
colazione con i dolci " camera da letto negli hotel periferici di questa citta...
la risposta e no non ci mettera piede.. " solo ".. " solo " le persone che
cura per la colazione e non l'italiano.. solo "? o " solo " era la cameriera
che mi ha preceduto con il " suo " solo "? ho risposto " solo " mi ha risposto
solo il... e'sicuro, manca l'accesso di vicini e wi - fi e.... in realta
chiedere se si spegne. le camere forniscono collegamenti con la camera tramite
la quale non avevo letto la prima ricerca..... camere charmese e
confortevoli con mo e bagno sporchi. il letto / wc sembrava un pole stile paradisia
che soggiornariati come servizi e altri servizi. la colazione a
sopportare l'

In [ ]:
%%time
parameters = {'n_sentences': 3,  
              'batch_size': 3,
              'avg_len':200,
              'max_len':1000,
              'std_len' : 30,
              'top_k': 100,
              'generation_method':'attention',
              'temperature': 1,
              'sample': True,
              'burnin': 900,
              'max_iter': 1000,
             'seed_text': "[pos-0] [pos-1] [pos-2] ",
              }


sents = it_bert_model.generate(save_to_path='/content/prova.txt', **parameters)

print_reviews(sents)


REVIEW 0
assolutamente molto deludente, con tutte le strutture una corsa,
senza raggiungere il porto. benche si sia rivelato molto invidiabile, il
prezzo e statodito. non capisco come sia lo spazio. da tralasciare la stanza
non super elegante, il bagno parole di lode. la colazione veramente
mediocre per la qualita degli. solo le 4 fette del piano cottura erano di pando e
di porchetta, invece anche le sfogliatelle per la pizza, ma con tutte
leta buone in piu la varieta di antipasti ma anche le mazzacene. le
cameriere fanno uso di un. il della colazione era, quasi scortese. si quando si
stendevano di quellene che di gamberi duri. piatti lunghi buffet freddi con una
piazzola di insalata dicciata per il resto si per scontatova latta che si erato
con un osso duro. non ci sono un posto, certo per il mare per spiaggia, per la
spiaggia. nota positiva la cena in spiaggia e il peggior di di un albergo. penso

REVIEW 1
dire che la grande cosa sia da parti, ma la mia cara recensione non male
mi s

In [ ]:
%%time
parameters = {'n_sentences': 3,  
              'batch_size': 3,
              'avg_len':200,
              'max_len':1000,
              'std_len' : 30,
              'top_k': 100,
              'generation_method':'attention',
              'temperature': 1,
              'sample': True,
              'burnin': 900,
              'max_iter': 1000,
             'seed_text': "[neg-0] [neg-1] [neg-2] ",
              }


sents = it_bert_model.generate(save_to_path='/content/prova.txt', **parameters)

print_reviews(sents)


REVIEW 0
per due volte meritava cinque stelle con solo poche bevande
chilometriche. tutto questo solo per il buon prezzo rispetto ad una struttura a 5....
questo hotel e davvero molto.. peccato che sono rimasta molto delusa, ho
dovuto una camera matrimoniale e ne avevo una doppia se non una sola e non era
presente nemmeno una struttura ad.... nel complesso tutto questo a
anche...... non si trova qualcuno che possa controllare il posizionamento del
letto o quello del letto.... senza. e stato veramente impossibile trovare
un hotel a 5 stelle... la mia esperienza deve essere trasformata piu un
hotel... e..... anche se si puo dire che in realta e un hotel a.... che dire.. non

REVIEW 1
soggiornato per un grand tour internazionale e, ripeto, altro
ancora. mi sembra essere molto soddisfatto della situazione. il grand hotel e
bellissimo, pulito e arredamento molto e un servizio adeguato........ e stato un
buon auspicio tutto quello che si e verificato con........... i canali
comunicanti del 

In [ ]:
' '.join(s.split()[:10])

'nn lo so non non accendo se veramente e necessario...'